In [1]:
%pip install -q python-terrier
%pip install -q --upgrade git+https://github.com/terrierteam/pyterrier_t5.git

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pyterrier as pt
pt.init()

PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [3]:
from pyterrier_t5 import MonoT5ReRanker, DuoT5ReRanker
monoT5 = MonoT5ReRanker(text_field='abstract')
duoT5 = DuoT5ReRanker(text_field='abstract')

In [4]:
import os
dataset = pt.get_dataset("irds:cord19/trec-covid")
index_loc = "./index_path"
indexer = pt.IterDictIndexer(index_loc, overwrite=True)
indexref = indexer.index(dataset.get_corpus_iter(), fields=('title', 'abstract'))

In [5]:
bm25 = pt.BatchRetrieve(indexref, wmodel="BM25") % 100
mono_pipeline = bm25 >> pt.text.get_text(dataset, "abstract") >> monoT5
duo_pipeline = mono_pipeline % 10 >> duoT5

In [6]:
pt.Experiment(
  [
   bm25,
   mono_pipeline,
   duo_pipeline,
  ],
  dataset.get_topics('title'),
  dataset.get_qrels(),
  names=["BM25", "BM25 >> monoT5", "BM25 >> monoT5 >> duoT5"],
  eval_metrics=["map", "recip_rank", "P.10", "ndcg_cut.10"]
)

duoT5: 100%|██████████| 50/50 [20:33<00:00, 24.66s/queries]


,name,map,recip_rank,P.10,ndcg_cut.10
0,BM25,0.076080,0.807094,0.678,0.601847
1,BM25 >> monoT5,0.081686,0.869000,0.742,0.677117
2,BM25 >> monoT5 >> duoT5,0.015155,0.838889,0.726,0.659669
